# Predicción del coste de un seguro
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 15-06-2023*<br>

https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
import seaborn as sns
%matplotlib inline

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'seaborn'

## 1 Análisis exploratorio de datos

In [ ]:
# importar datos
insurance = pd.read_csv('../data/insurance.csv')

insurance.info()

Hay al menos 3 variables categóricos (_sex_, _smoker_, _region_), puesto que son de tipo texto. Además, incluiré como categórica la variable _children_, puesto que, a pesar de ser de tipo entero, tiene valores discretos que pueden tomarse como categorías. Sobre estas categorías habrá que hacer un preprocesado para que puedan utilizarse en el modelo de regresión.

Por el contrario, las variables _age_ y _bmi_ son numéricas. Sobre ellas aplicaré un escalado para que tengan un orden de magnitud similar.

In [ ]:
# check if there are missing values in the dataset
insurance.isnull().sum()

El resultado de la celda anterior muestra que no hay datos faltantes que sea necesario rellenar en el preprocesado de los datos.

Ahora genero dos gráficos para tener una idea de la relación entre las variables. Primero genero un gráfico con la matriz de correlación. Luego genero un gráfico que incluye los diagramas de dispersión de cada par de variables y la función de distribución de cada variable. Para que las variables categóricas aparezcan en este segundo gráfico tengo que convertirlas en numéricas "manualmente".

In [ ]:
insurance_transformed = insurance.replace({'sex': {'female': 1, 'male': 0},
                                           'smoker': {'yes': 1, 'no': 0},
                                           'region': {'southwest': 2, 'southeast': 1, 'northwest': 3, 'northeast': 0}})

In [ ]:
# matriz de correlación
sns.heatmap(insurance_transformed.corr('spearman') > .5, cmap='viridis');

In [ ]:
sns.pairplot(insurance_transformed, hue='smoker', corner=True);

Se ve una clara relación entre el coste del seguro y el hecho de ser fumador o no.Además, hay un cierto incremento del coste con la edad y el índice de masa corporal (aunque esta última no aparece en la matriz de correlación). A priori no se ve una relación clara ni con el sexo ni la región.

## 2 Modelo de predicción

He decidido utilizar como modelo de regresión `SGDRegressor`. Antes de poder aplicar el modelo, creo un preprocesador independiente para las variables numéricas y las variables categóricas, y los uno en un único `Pipeline` de preprocesado. Posteriormente creo el `Pipeline` del modelo, que incluye el preprocesador y el modelo de regresión.

Entreno el modelo y veo el rendimiento tanto en la muestra de entrenamiento como en la de validación. He utilizado como métrica la raíz del error cuadrático medio (RMSE) porque tiene las mismas unidades que la variable a predecir.

In [10]:
# crear muestras de entrenamiento y validación
y = insurance.charges
X = insurance.drop('charges', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0, shuffle=True)

In [13]:
# extortar muestras de entrenamiento y validación
train = pd.concat((X_train, y_train), axis=1)
train.to_csv('my_model/datasets/train.csv', index=False)
test = pd.concat((X_test, y_test), axis=1)
test.to_csv('my_model/datasets/test.csv', index=False)

In [ ]:
# preprocesador de las variables numéricas: reescalado de las variables
numeric_features = ['age', 'bmi']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# preprocesador de las variables categóricas: one hot encoder
categorical_features = ['sex', 'smoker', 'region', 'children']
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore'))])

# unir ambos preprocesos en un solo transformador
preprocessor = ColumnTransformer(transformers=[('numeric', numeric_transformer, numeric_features),
                                               ('categorical', categorical_transformer, categorical_features)])

# definir el pipeline del modelo: preproceso + regresor
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', SGDRegressor(max_iter=1000, penalty='elasticnet', tol=1e-3, random_state=0))])

In [ ]:
model.fit(X_train, y_train.values)

In [ ]:
# calcular el rendimiento del modelo como la raíz del error cuadrático medio
rmse_train = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
rmse_test = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
print('RMSE (train) =\t{0:.2f} €'.format(rmse_train))
print('RMSE (test)  =\t{0:.2f} €'.format(rmse_test))

In [ ]:
# exportar el modelo
joblib.dump(model, open('pl_insurance.pkl', 'wb'))